<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%9D%B4%EC%86%8C%ED%98%84_%EC%84%9C%EB%B9%84%EC%8A%A42_GRU%EA%B8%B0%EB%B0%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset

from sklearn.utils import resample

import warnings
%config InlineBackend.figure_format = 'retina'
warnings.filterwarnings("ignore")
%matplotlib inline

In [4]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os
import re

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/Final_Project/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [5]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.5 MB/s eta 0:00:00


# 요기요 데이터셋 불러오기

In [61]:
train = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0908.csv")
train.head()

,가게이름,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글,메뉴
0,1988응답하라추억의옛날도시락-방이점,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,5,5.0,5.0,5.0,즐거운 순간을 저희와 함께해주셔서 감사합니다💖 \n\n보람차게 일할 수 있게 해 주...,[직화]삼겹옛날도시락
1,1988응답하라추억의옛날도시락-방이점,기름 있는 부분이 생각보다 많아요! 참고하세요:),5,5.0,5.0,5.0,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,"[직화]삼겹옛날도시락,볶음김치"
2,1988응답하라추억의옛날도시락-방이점,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,5,5.0,5.0,5.0,늘 최고의 메뉴를 제공해드리고 싶습니다!😄 \n\n맛있는 메뉴를 넉넉하게 먹을 수 ...,"[직화]삼겹옛날도시락,우삼겹청국장찌개＋대접밥"
3,1988응답하라추억의옛날도시락-방이점,"..,........",5,5.0,5.0,5.0,저희 매장과 함께 편안한 시간 되셨을까요? \n\n1. 고객을 생각하는 마음 \n2...,삼겹정찬도시락
4,1988응답하라추억의옛날도시락-방이점,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,5,5.0,4.0,5.0,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요!😁\n\n고객님의 행복...,"갈비옛날도시락＋미니냉면＋（서비스）콜라,스팸옛날도시락＋김찌 정식＋（서비스）콜라,부드러..."


In [62]:
for i in range(1, 6):
    num_rating = train[train['별점'] == i].shape[0]
    print(f"별점 {i}점 개수: {num_rating}, {num_rating / train.shape[0] * 100}%")

별점 1점 개수: 220, 0.5923532579429186%
별점 2점 개수: 137, 0.36887452880990845%
별점 3점 개수: 385, 1.0366182014001075%
별점 4점 개수: 1234, 3.3225632740980076%
별점 5점 개수: 35164, 94.67959073774905%


# 데이터 전처리

## 1_특수문자 제거

In [64]:
review_list = []
for review in train['고객리뷰']:
    if isinstance(review, str):
        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", review)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['고객리뷰'] = review_list
train.head()

,가게이름,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글,메뉴
0,1988응답하라추억의옛날도시락-방이점,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,5,5.0,5.0,5.0,즐거운 순간을 저희와 함께해주셔서 감사합니다💖 \n\n보람차게 일할 수 있게 해 주...,[직화]삼겹옛날도시락
1,1988응답하라추억의옛날도시락-방이점,기름 있는 부분이 생각보다 많아요! 참고하세요,5,5.0,5.0,5.0,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,"[직화]삼겹옛날도시락,볶음김치"
2,1988응답하라추억의옛날도시락-방이점,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,5,5.0,5.0,5.0,늘 최고의 메뉴를 제공해드리고 싶습니다!😄 \n\n맛있는 메뉴를 넉넉하게 먹을 수 ...,"[직화]삼겹옛날도시락,우삼겹청국장찌개＋대접밥"
3,1988응답하라추억의옛날도시락-방이점,".,.",5,5.0,5.0,5.0,저희 매장과 함께 편안한 시간 되셨을까요? \n\n1. 고객을 생각하는 마음 \n2...,삼겹정찬도시락
4,1988응답하라추억의옛날도시락-방이점,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,5,5.0,4.0,5.0,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요!😁\n\n고객님의 행복...,"갈비옛날도시락＋미니냉면＋（서비스）콜라,스팸옛날도시락＋김찌 정식＋（서비스）콜라,부드러..."


In [65]:
reply_list = []
for reply in train['사장댓글']:
    if isinstance(reply, str):
        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", reply)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['사장댓글'] = reply_list
train.head()

,가게이름,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글,메뉴
0,1988응답하라추억의옛날도시락-방이점,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,5,5.0,5.0,5.0,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,[직화]삼겹옛날도시락
1,1988응답하라추억의옛날도시락-방이점,기름 있는 부분이 생각보다 많아요! 참고하세요,5,5.0,5.0,5.0,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,"[직화]삼겹옛날도시락,볶음김치"
2,1988응답하라추억의옛날도시락-방이점,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,5,5.0,5.0,5.0,늘 최고의 메뉴를 제공해드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,"[직화]삼겹옛날도시락,우삼겹청국장찌개＋대접밥"
3,1988응답하라추억의옛날도시락-방이점,".,.",5,5.0,5.0,5.0,저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,삼겹정찬도시락
4,1988응답하라추억의옛날도시락-방이점,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,5,5.0,4.0,5.0,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,"갈비옛날도시락＋미니냉면＋（서비스）콜라,스팸옛날도시락＋김찌 정식＋（서비스）콜라,부드러..."


##2_고객ID -> 고객님

In [66]:
train['고객리뷰'] = train['고객리뷰'].astype(str)
train['사장댓글'] = train['사장댓글'].astype(str)

In [67]:
train['사장댓글'] = train['사장댓글'].str.replace(r'\w+\s* 님', '고객님', regex=True)

##3_가게명 키워드 컬럼화

In [68]:
store_list = train['가게이름']
store_list = (store_list.str.split('-')).str.get(0)
store_list

0        1988응답하라추억의옛날도시락
1        1988응답하라추억의옛날도시락
2        1988응답하라추억의옛날도시락
3        1988응답하라추억의옛날도시락
4        1988응답하라추억의옛날도시락
               ...       
37135                 짬뽕관
37136                 짬뽕관
37137                 짬뽕관
37138                 짬뽕관
37139                 짬뽕관
Name: 가게이름, Length: 37140, dtype: object

In [72]:
tmp = list(set(store_list.replace("[^가-힣a-zA-Z0-9 ]"," ",regex=True)))
tmp

['왕빈자삼파전',
 '잭아저씨족발보쌈',
 '원할머니보쌈',
 '모어댄버터',
 '국수나무',
 '우리집반찬도시락',
 '삽교원조두리곱창',
 '코브라독스',
 '부산오빠1인국밥',
 '짬뽕관',
 '손이가시래기감자탕',
 '천년닭강정',
 '땅끝수산',
 '삼다수',
 '조형훈족발보쌈',
 '울진죽변항',
 '랜돌프뉴욕페페로니',
 '인생제육',
 '더티베지',
 '알촌',
 '걸작떡볶이',
 '짱닭치킨',
 '빕스얌딜리버리',
 '우시장국밥',
 '구구족',
 '보돌미역',
 '수제죽전문점',
 '최고남제육명가',
 '백소정',
 '어시장',
 '윤희횟집',
 '진지덮밥',
 '탕화쿵푸 마라탕 가오점',
 '라화방마라탕',
 '홍스족발',
 '수피자',
 '승도리네곱도리탕',
 '도야짬뽕 계림점',
 '쇼부덮밥',
 '냉면 숯불고기전문점줄벼락',
 '배달의쌀국수',
 '짚신스시',
 '나인곱창',
 '청춘식당묵은지김치찜',
 '폴트버거',
 '신자매김치찜 김치찌개',
 '띵하오차이',
 '카페봄봄',
 '1988응답하라추억의옛날도시락',
 '빽다방',
 '최가네한쌈',
 '1989마라탕',
 '고씨네카레',
 '이토우야',
 '용문',
 '탐나는피자',
 '덮덮밥',
 '플러스82',
 '부성초밥',
 '창타이누들',
 '콩샌전북대점',
 '행복한찜닭',
 '고기듬뿍미트박스',
 '누보닭갈비',
 '초밥대통령',
 '후스폭립',
 '이화수육개장',
 '짜글이가돼었소',
 '인생아구찜',
 '미락칼국수',
 '부대찌개자신있는집',
 'OK반점',
 '올데이케밥 샐러드',
 '텐텐마라탕',
 '뜸들이다',
 '황궁쟁반짜장',
 '빅스타피자',
 '땡초곱창막창',
 '본점 장충왕족발보쌈',
 '수락',
 '놀부 부대찌개',
 '대찌 대파생고기김치찌개 정육왕국물두루치기',
 '상무초밥',
 '청년치킨',
 '무국적식탁',
 '스시리',
 '도우라이크피자',
 '고기혁명',
 '고돼지',
 '이끌림마라탕',
 '롱쓰부대찌개',
 '놀부보쌈족발',
 '비오키친',


- 띄어쓰기 있을 경우 띄어쓰기 없앤 가게명 (ex)놀부부대찌개

In [73]:
tmp2 = [''.join(store.split()) for store in tmp]
tmp2

['왕빈자삼파전',
 '잭아저씨족발보쌈',
 '원할머니보쌈',
 '모어댄버터',
 '국수나무',
 '우리집반찬도시락',
 '삽교원조두리곱창',
 '코브라독스',
 '부산오빠1인국밥',
 '짬뽕관',
 '손이가시래기감자탕',
 '천년닭강정',
 '땅끝수산',
 '삼다수',
 '조형훈족발보쌈',
 '울진죽변항',
 '랜돌프뉴욕페페로니',
 '인생제육',
 '더티베지',
 '알촌',
 '걸작떡볶이',
 '짱닭치킨',
 '빕스얌딜리버리',
 '우시장국밥',
 '구구족',
 '보돌미역',
 '수제죽전문점',
 '최고남제육명가',
 '백소정',
 '어시장',
 '윤희횟집',
 '진지덮밥',
 '탕화쿵푸마라탕가오점',
 '라화방마라탕',
 '홍스족발',
 '수피자',
 '승도리네곱도리탕',
 '도야짬뽕계림점',
 '쇼부덮밥',
 '냉면숯불고기전문점줄벼락',
 '배달의쌀국수',
 '짚신스시',
 '나인곱창',
 '청춘식당묵은지김치찜',
 '폴트버거',
 '신자매김치찜김치찌개',
 '띵하오차이',
 '카페봄봄',
 '1988응답하라추억의옛날도시락',
 '빽다방',
 '최가네한쌈',
 '1989마라탕',
 '고씨네카레',
 '이토우야',
 '용문',
 '탐나는피자',
 '덮덮밥',
 '플러스82',
 '부성초밥',
 '창타이누들',
 '콩샌전북대점',
 '행복한찜닭',
 '고기듬뿍미트박스',
 '누보닭갈비',
 '초밥대통령',
 '후스폭립',
 '이화수육개장',
 '짜글이가돼었소',
 '인생아구찜',
 '미락칼국수',
 '부대찌개자신있는집',
 'OK반점',
 '올데이케밥샐러드',
 '텐텐마라탕',
 '뜸들이다',
 '황궁쟁반짜장',
 '빅스타피자',
 '땡초곱창막창',
 '본점장충왕족발보쌈',
 '수락',
 '놀부부대찌개',
 '대찌대파생고기김치찌개정육왕국물두루치기',
 '상무초밥',
 '청년치킨',
 '무국적식탁',
 '스시리',
 '도우라이크피자',
 '고기혁명',
 '고돼지',
 '이끌림마라탕',
 '롱쓰부대찌개',
 '놀부보쌈족발',
 '비오키친',
 '오늘은분식',


- 띄어쓰기한 가게명

In [19]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-_s1grwsx
  Running command git clone --filter=blob:none --quiet https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-_s1grwsx
  Resolved https://github.com/ssut/py-hanspell.git to commit fdc6ca50c19f1c85971437a072d89d4e5ce024b8
  Preparing metadata (setup.py) ... done
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4813 sha256=2571bc35bd5167690a2b13fba26e00c6ffaf3269e23062227b9096dc0d2ccefb
  Stored in directory: /tmp/pip-ephem-wheel-cache-o7jittd6/wheels/2e/43/cc/753c9e1d91affb9ea40e186cea5654fb9231deb454da6724e5
Successfully built py-hanspell


In [74]:
from hanspell import spell_checker

corrected_store = []
for name in tmp:
    hanspell_sent = spell_checker.check(name)
    corrected_store.append(hanspell_sent.checked)

corrected_store

['왕 빈자 삼파전',
 '잭 아저씨 족발보쌈',
 '원 할머니보쌈',
 '모어 덴버 터',
 '국수나무',
 '우리 집 반찬도시락',
 '삽교 원조 두리 곱창',
 '코브라 독습',
 '부산 오빠 1인 국밥',
 '짬뽕과',
 '손이 가시에 기 감자탕',
 '천년 닭강정',
 '땅끝 수산',
 '삼다수',
 '조형 훈 족발보쌈',
 '울진 죽변항',
 '랜돌프 뉴욕 페퍼로니',
 '인생 제육',
 '더티 베지',
 '알촌',
 '걸작 떡볶이',
 '장닭 치킨',
 '빕스얌딜리버리',
 '우시장 국밥',
 '구구족',
 '보도로 미역',
 '수제 죽 전문점',
 '최고 날제육 명가',
 '백 소장',
 '어시장',
 '윤희 횟집',
 '진지 덮밥',
 '탕화 쿵후 마라탕 가오점',
 '라화방마라탕',
 '홍수 족발',
 '수비자',
 '승도리네곱도리탕',
 '도야 짬뽕 계림점',
 '승부 덮밥',
 '냉면 숯불고기전문점줄벼락',
 '배달의 쌀국수',
 '짚신 스시',
 '나인 곱창',
 '청춘 식당 묵은 지 김치찜',
 '폴트 버거',
 '신 자매 김치찜 김치찌개',
 '띵하오 차이',
 '카페 봄봄',
 '1988응답하라 추억의 옛날 도시락',
 '빽다방',
 '최가네 한 쌈',
 '1989마라탕',
 '고씨네 카레',
 '이토야',
 '용문',
 '탐나는 피자',
 '덮덮밥',
 '플러스 82',
 '부성 초밥',
 '창 타이 누들',
 '콩 샌 전북대점',
 '행복한 찜닭',
 '고기 듬뿍 미트 박스',
 '누보 닭갈비',
 '초밥 대통령',
 '후스 폭립',
 '이화수 육개장',
 '짜 글이가 되었소',
 '인생 아귀찜',
 '미락 칼국수',
 '부대찌개 자신 있는 집',
 'OK 반점',
 '올 데이 케밥 샐러드',
 '텐텐 마라탕',
 '뜸 들이다',
 '황궁쟁반짜장',
 '빅스타 피자',
 '땡초 곱창 막창',
 '본점 장충 왕족발보쌈',
 '수락',
 '놀부 부대찌개',
 '대치 대파 생고기 김치찌개 정육 왕국 물 두루치기',
 '상무 초밥',


- 띄어쓰기 기준으로 음식점 이름 쪼개보기

In [75]:
short_name = []
for store in corrected_store:
    tmp = store.split(' ')
    short_name.extend(tmp)

short_name = list(set(short_name)) # 중복 제거

In [76]:
len(short_name)

332

- 쪼개진 short_name 확인해서 음식점 이름으로 쓰일만한 것 남기기

In [77]:
short_name = ['두리',
 '구구족',
 '랜돌프',
 '글이가',
 '팔당',
 '부성',
 '빈자',
 '베지',
 '시나',
 '알촌',
 '승도리네곱도리탕',
 '스웨이',
 '삼파전',
 '홍싸롱',
 '프레드',
 '빕스얌딜리버리',
 '피제이',
 '깨돌이',
 '소빅',
 '이토야',
 '춘리',
 '프라',
 '삽교',
 'GTS',
 '윤희',
 '1989마라탕',
 '써브웨이',
 '날제육',
 '냉면숯불고기전문점줄벼락',
 '탕화',
 '미락',
 '덮덮밥',
 '살만',
 '소우',
 '스시가',
 '폴트',
 '프레',
 '마라의신마라탕마라샹궈',
 '마라하오',
 '이태리면',
 '백순대',
 '이화수',
 '후스',
 '황궁쟁반짜장',
 '국수나무',
 '카산도',
 '박가',
 '덴버',
 '텔로',
 '고씨네',
 '우시장',
 '할머니보쌈',
 '독습',
 '오키친',
 '도야',
 '39도시락',
 '빽다방',
 '보도로',
 '라화방마라탕',
 '수비자',
 '뭇국적',
 '빅스타',
 '라이크',
 '죽변항',
 '손수경의',
 '용문',
 '깨비',
 '텐텐',
 '띵하오',
 '당치',
 '놀부보쌈족발',
 '1988응답하라',
 '최가네',
 '조형',
 '만다린',
 '7번가',
 '봄봄',
 '전금례닭볶음탕',
 '밈이',
 '쿵후',
 '누보',
 '글이',
 '워커스']

In [78]:
store = tmp + corrected_store + tmp2 + short_name

In [79]:
store_list = list(set(store))
store_list

['짚신 스시',
 '모어댄버터',
 '고 돼지',
 '모어 덴버 터',
 '토핑 폭탄 김치찜 김치찌개',
 '손이가시래기감자탕',
 '저팔계중국마라탕덮밥요리전문점',
 '베지',
 '진지 덮밥',
 '알촌',
 '오늘은분식',
 '걸작떡볶이',
 '구구족',
 '봉명동 내 커피',
 '보돌미역',
 '미친 피자',
 '수제죽전문점',
 '스시가 소우',
 '직화 삼겹 직구 삼',
 '진지덮밥',
 '인생 제육',
 '천년 닭강정',
 '홍스족발',
 '수피자',
 '승도리네곱도리탕',
 '냉면 숯불고기전문점줄벼락',
 '이배달마약삼겹떡볶이상당구점',
 '이화수 육개장',
 '과일에 반하다',
 '프레드 피자',
 '깨돌이',
 '수제 죽 전문점',
 '청춘 식당 묵은 지 김치찜',
 '날제육',
 '탕화 쿵후 마라탕 가오점',
 '한식세끼1인김치찜김치찌개',
 '부산 오빠 1인 국밥',
 '최가네한쌈',
 '1989마라탕',
 '올 데이 케밥 샐러드',
 '플러스82',
 '땡초 곱창 막창',
 '대치 대파 생고기 김치찌개 정육 왕국 물 두루치기',
 '폴트 버거',
 '워커스',
 '뭇국적 식탁',
 '워커스커피로스터스',
 '초밥대통령',
 '후스폭립',
 '고씨네',
 '시나',
 '부대찌개 자신 있는 집',
 '홍대 개미 ',
 '코브라 독습',
 '조형 훈 족발보쌈',
 '청년 치킨',
 '빅스타피자',
 '구래 상회',
 '놀부 부대찌개',
 '플러스 82',
 '김포 불 닭발',
 '땡초곱창막창',
 '무국적식탁',
 '도우라이크피자',
 '고기혁명',
 '띵하오 차이',
 '비오키친',
 '오늘은 분식',
 '텔로',
 '카산도',
 '떡형',
 '춘리 마라탕',
 '피제이 피자',
 '짜 글이가 되었소',
 '승부 덮밥',
 '빅스타',
 '혼밥 대왕',
 '박가 부대',
 '짜',
 '프레드피자',
 '깨돌이김밥',
 '7번가 피자',
 '피제이',
 '순살만공격',
 '텐텐',
 '랜돌프 뉴욕 페퍼로니',
 '유쾌한꼬막한끼끝판왕',
 '홍

- 빠진 음식점 이름 추가하기

In [80]:
store_add = ['당신은 지금 치킨이 땡긴다','당신은 지금 치킨이 당긴다','마라 하오','빕스 얌 딜리버리','한식 끼','헬로팬','라화방','미.친피자','밈 피자','부자집','안녕닭','토핑폭탄','카마쿠라','땡초곱창','이배달','프루타','시민족발보쌈','조형훈의족발보쌈도시락','잭아저씨 족발보쌈']

In [81]:
store_list.extend(store_add)

In [82]:
train['store'] = train['사장댓글'].apply(lambda x: next((store for store in store_list if store in x), " "))

In [83]:
train.head()

,가게이름,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글,메뉴,store
0,1988응답하라추억의옛날도시락-방이점,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,5,5.0,5.0,5.0,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,[직화]삼겹옛날도시락,
1,1988응답하라추억의옛날도시락-방이점,기름 있는 부분이 생각보다 많아요! 참고하세요,5,5.0,5.0,5.0,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,"[직화]삼겹옛날도시락,볶음김치",
2,1988응답하라추억의옛날도시락-방이점,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,5,5.0,5.0,5.0,늘 최고의 메뉴를 제공해드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,"[직화]삼겹옛날도시락,우삼겹청국장찌개＋대접밥",
3,1988응답하라추억의옛날도시락-방이점,".,.",5,5.0,5.0,5.0,저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,삼겹정찬도시락,
4,1988응답하라추억의옛날도시락-방이점,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,5,5.0,4.0,5.0,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,"갈비옛날도시락＋미니냉면＋（서비스）콜라,스팸옛날도시락＋김찌 정식＋（서비스）콜라,부드러...",


##4_지점명 키워드 컬럼화

In [84]:
store_list = train['가게이름']
branch_arr = (store_list.str.split('-| ')).str.get(1).unique()
branch_arr

array(['방이점', '로스터스', '가락점', '헬리오시티점', '석촌점', '송파점', nan, '삼전점', '본점',
       '미아본점', '성신여대역점', '대학로점', '성북점', '동대문구점', '북서울꿈의숲점', '김치찜',
       '혜화막내딸지점', '대파생고기김치찌개&정육왕국물두루치기', '부천점', '부평점', '부천부평점', '본가진죽',
       '분식', '신중동점', '부천시청점', '도촌점', '야탑점', '판교점', '수제돈까스&파스타', '정자본점',
       '미금본점', '미금점', '분당정자점', '분당미금역점', '죽전점', '용인수지점', '카마쿠라', '서현본점',
       '분당점', '', '용암점', '마약삼겹떡볶이', '대전4호점', '신탄진점', '신일점', '목동충촌점',
       '대전동구점', '우송대점', '대전역점', '대흥문창점', '서대전점', '유성봉명점', '장충왕족발보쌈', '대전',
       '대덕테크노벨리점', '대전전민점', '마라탕', '대전한남대점', '충남대점', '유성점', '광주백운점',
       '계림점', '조대점', '광주본점', '광주광천점', '북구(전대)점', '광주직영점', '부대찌개', '전주평화점',
       '객사점', '전주1호점', '전주금암점'], dtype=object)

In [85]:
branch_lst = [item for item in branch_arr if isinstance(item, str) and item.endswith('점')]
branch_lst

['방이점',
 '가락점',
 '헬리오시티점',
 '석촌점',
 '송파점',
 '삼전점',
 '본점',
 '미아본점',
 '성신여대역점',
 '대학로점',
 '성북점',
 '동대문구점',
 '북서울꿈의숲점',
 '혜화막내딸지점',
 '부천점',
 '부평점',
 '부천부평점',
 '신중동점',
 '부천시청점',
 '도촌점',
 '야탑점',
 '판교점',
 '정자본점',
 '미금본점',
 '미금점',
 '분당정자점',
 '분당미금역점',
 '죽전점',
 '용인수지점',
 '서현본점',
 '분당점',
 '용암점',
 '대전4호점',
 '신탄진점',
 '신일점',
 '목동충촌점',
 '대전동구점',
 '우송대점',
 '대전역점',
 '대흥문창점',
 '서대전점',
 '유성봉명점',
 '대덕테크노벨리점',
 '대전전민점',
 '대전한남대점',
 '충남대점',
 '유성점',
 '광주백운점',
 '계림점',
 '조대점',
 '광주본점',
 '광주광천점',
 '북구(전대)점',
 '광주직영점',
 '전주평화점',
 '객사점',
 '전주1호점',
 '전주금암점']

- 지점명에 띄어쓰기가 있을 경우

In [86]:
branch_space = []
for name in branch_lst:
    hanspell_sent = spell_checker.check(name)
    branch_space.append(hanspell_sent.checked)

branch_space

['방이점',
 '가락점',
 '헬리오 시티점',
 '석촌점',
 '송파점',
 '삼전점',
 '본점',
 '미아 본점',
 '성신여대역점',
 '대학로점',
 '성북점',
 '동대문구점',
 '북서울꿈의 숲점',
 '혜화 막내딸 지점',
 '부천점',
 '부평점',
 '부천 부평점',
 '신중동점',
 '부천 시청점',
 '도촌점',
 '야탑점',
 '판교점',
 '정자본점',
 '미금 본점',
 '미금점',
 '분당 정자점',
 '분당 미금역점',
 '죽전점',
 '용인 수지점',
 '서현 본점',
 '분당점',
 '용암점',
 '대전 4호점',
 '신탄진점',
 '신일점',
 '목동충촌점',
 '대전 동구점',
 '우송대점',
 '대전역점',
 '대흥 문창점',
 '서대전점',
 '유성 봉명점',
 '대덕테크노벨리점',
 '대전 전민점',
 '대전 한남대점',
 '충남대점',
 '유성점',
 '광주백운점',
 '계림점',
 '조대점',
 '광주 본점',
 '광주 광천점',
 '북구(전대) 점',
 '광주 직영점',
 '전주 평화점',
 '객사점',
 '전주 1호점',
 '전주 금암점']

- 띄어쓰기 적용한 가게명 중 '점' 찾기 (ex- 콩 샌 전북대점)

In [87]:
branch = []
for i in corrected_store:
    if i.endswith('점'):
        branch.append(i)
branch

['수제 죽 전문점',
 '탕화 쿵후 마라탕 가오점',
 '도야 짬뽕 계림점',
 '콩 샌 전북대점',
 'OK 반점',
 '이 배달 마약 삼겹 떡볶이 상당구점',
 '맘스터치 대덕테크노벨리점',
 '저팔계 중국 마라탕 덮밥 요리 전문점',
 '닭 강점']

In [88]:
branch_space.append('전북대점')

In [89]:
branch_list = branch_lst + branch_space

In [90]:
branch_list = list(set(branch_list)) # 중복 제거
branch_list

['광주 본점',
 '용인수지점',
 '대전 한남대점',
 '부천 시청점',
 '송파점',
 '전북대점',
 '미아본점',
 '목동충촌점',
 '석촌점',
 '미금본점',
 '서현 본점',
 '대전동구점',
 '부천시청점',
 '가락점',
 '동대문구점',
 '분당정자점',
 '미금 본점',
 '본점',
 '대전한남대점',
 '유성점',
 '광주직영점',
 '전주1호점',
 '도촌점',
 '전주 1호점',
 '신일점',
 '대전 4호점',
 '죽전점',
 '혜화 막내딸 지점',
 '삼전점',
 '대전역점',
 '미아 본점',
 '야탑점',
 '전주 금암점',
 '북서울꿈의숲점',
 '유성 봉명점',
 '북서울꿈의 숲점',
 '전주 평화점',
 '전주금암점',
 '분당 정자점',
 '북구(전대)점',
 '판교점',
 '광주 직영점',
 '대학로점',
 '대전4호점',
 '계림점',
 '서대전점',
 '광주백운점',
 '광주 광천점',
 '부천 부평점',
 '대전전민점',
 '정자본점',
 '전주평화점',
 '분당 미금역점',
 '용암점',
 '혜화막내딸지점',
 '대전 동구점',
 '성북점',
 '충남대점',
 '우송대점',
 '대전 전민점',
 '서현본점',
 '부평점',
 '대덕테크노벨리점',
 '신탄진점',
 '신중동점',
 '광주본점',
 '성신여대역점',
 '대흥문창점',
 '부천부평점',
 '분당미금역점',
 '헬리오 시티점',
 '부천점',
 '헬리오시티점',
 '객사점',
 '광주광천점',
 '조대점',
 '분당점',
 '방이점',
 '용인 수지점',
 '미금점',
 '유성봉명점',
 '북구(전대) 점',
 '대흥 문창점']

In [91]:
train['branch'] = train['사장댓글'].apply(lambda x: next((branch for branch in branch_list if branch in x), " "))
train['branch']

0         
1         
2         
3         
4         
        ..
37135     
37136     
37137     
37138     
37139     
Name: branch, Length: 37140, dtype: object

In [92]:
train.head()

,가게이름,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글,메뉴,store,branch
0,1988응답하라추억의옛날도시락-방이점,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,5,5.0,5.0,5.0,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,[직화]삼겹옛날도시락,,
1,1988응답하라추억의옛날도시락-방이점,기름 있는 부분이 생각보다 많아요! 참고하세요,5,5.0,5.0,5.0,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,"[직화]삼겹옛날도시락,볶음김치",,
2,1988응답하라추억의옛날도시락-방이점,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,5,5.0,5.0,5.0,늘 최고의 메뉴를 제공해드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,"[직화]삼겹옛날도시락,우삼겹청국장찌개＋대접밥",,
3,1988응답하라추억의옛날도시락-방이점,".,.",5,5.0,5.0,5.0,저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,삼겹정찬도시락,,
4,1988응답하라추억의옛날도시락-방이점,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,5,5.0,4.0,5.0,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,"갈비옛날도시락＋미니냉면＋（서비스）콜라,스팸옛날도시락＋김찌 정식＋（서비스）콜라,부드러...",,


##5_음식 키워드 컬럼화

In [41]:
food_list = pd.read_csv(f"{DATA_PATH}food.csv").drop_duplicates(ignore_index = True)["대표식품명"]
food_list

0          케이크
1          치즈빵
2         카스텔라
3          츄러스
4         치아바타
        ...   
753      햄채소볶음
754    무생채(무만)
755     참나물겉절이
756     치커리겉절이
757       짜장소스
Name: 대표식품명, Length: 758, dtype: object

In [42]:
food_lst = []
for food in food_list:
    parts = re.split('/|-|\(', food)
    food_lst.append(parts[0])

In [44]:
food_lst

['케이크',
 '치즈빵',
 '카스텔라',
 '츄러스',
 '치아바타',
 '와플',
 '찰떡빵',
 '김밥',
 '덮밥',
 '보리밥',
 '볶음밥',
 '비빔밥',
 '삼각김밥',
 '수수밥',
 '알밥',
 '영양돌솥밥',
 '오므라이스',
 '자장밥',
 '잡곡밥',
 '잡채밥',
 '주먹밥',
 '짬뽕밥',
 '차조밥',
 '찰밥',
 '초밥',
 '카레라이스',
 '콩나물밥',
 '콩밥',
 '하이라이스',
 '현미밥',
 '회덮밥',
 '가래떡',
 '경단',
 '꿀떡',
 '모듬찰떡',
 '백설기',
 '송편',
 '시루떡',
 '약식',
 '인절미',
 '절편',
 '증편',
 '찹쌀떡',
 '도넛',
 '마늘빵',
 '머핀',
 '모닝빵',
 '모카빵',
 '베이글',
 '샌드위치',
 '소보로빵',
 '식빵',
 '초코소라빵',
 '크로와상',
 '파이',
 '페이스트리',
 '피자',
 '피자빵',
 '햄버거',
 '호떡',
 '간자장',
 '국수',
 '냉면',
 '떡국',
 '떡만두국',
 '라면',
 '만두',
 '만두국',
 '메밀국수',
 '수제비',
 '스파게티',
 '우동',
 '우동볶음',
 '월남쌈',
 '자장면',
 '짬뽕',
 '쫄면',
 '칼국수',
 '콩국수',
 '닭죽',
 '스프',
 '오리죽',
 '전복죽',
 '채소죽',
 '팥죽',
 '호박죽',
 '흑임자죽',
 '갈비탕',
 '감자국',
 '곰탕',
 '굴국',
 '김치국',
 '내장탕',
 '냉국',
 '달걀국',
 '달걀탕',
 '닭곰탕',
 '도가니탕',
 '된장국',
 '무국',
 '묵국',
 '미역국',
 '배추국',
 '북어국',
 '사골국',
 '삼계탕',
 '새우탕',
 '설렁탕',
 '순대국',
 '어묵국',
 '오징어국',
 '육개장',
 '장어탕',
 '재첩국',
 '조개국',
 '콩나물국',
 '토란국',
 '해장국',
 '갈치찌개',
 '감자탕',
 '김치전골',
 '김치찌개',
 '꽃게탕',
 '낙지전골',
 '동태국',
 '동

In [43]:
food_tmp = []
for name in food_lst:
    hanspell_sent = spell_checker.check(name)
    food_tmp.append(hanspell_sent.checked)

food_tmp

['케이크',
 '치즈빵',
 '카스텔라',
 '추로스',
 '치아바타',
 '와플',
 '찰떡 빵',
 '김밥',
 '덮밥',
 '보리밥',
 '볶음밥',
 '비빔밥',
 '삼각김밥',
 '수수밥',
 '알밥',
 '영양 돌솥밥',
 '오므라이스',
 '자장밥',
 '잡곡밥',
 '잡채밥',
 '주먹밥',
 '짬뽕밥',
 '차조밥',
 '찰밥',
 '초밥',
 '카레라이스',
 '콩나물밥',
 '콩밥',
 '하이라이스',
 '현미밥',
 '회덮밥',
 '가래떡',
 '경단',
 '꿀떡',
 '모둠 찰떡',
 '백설기',
 '송편',
 '시루떡',
 '약식',
 '인절미',
 '절편',
 '증편',
 '찹쌀떡',
 '도넛',
 '마늘빵',
 '머핀',
 '모닝빵',
 '모카빵',
 '베이글',
 '샌드위치',
 '곰보빵',
 '식빵',
 '초코 소라고 빵',
 '크루아상',
 '파이',
 '페이스트리',
 '피자',
 '피자빵',
 '햄버거',
 '호떡',
 '간자장',
 '국수',
 '냉면',
 '떡국',
 '떡만둣국',
 '라면',
 '만두',
 '만둣국',
 '메밀국수',
 '수제비',
 '스파게티',
 '가락국수',
 '가락국수 볶음',
 '월남쌈',
 '자장면',
 '짬뽕',
 '쫄면',
 '칼국수',
 '콩국수',
 '닭죽',
 '수프',
 '오리 죽',
 '전복죽',
 '채소 죽',
 '팥죽',
 '호박죽',
 '흑임자죽',
 '갈비탕',
 '감잣국',
 '곰탕',
 '굴국',
 '김칫국',
 '내장탕',
 '냉국',
 '달 결국',
 '달걀탕',
 '닭곰탕',
 '도가니탕',
 '된장국',
 '뭇국',
 '묵국',
 '미역국',
 '배춧국',
 '북엇국',
 '사골국',
 '삼계탕',
 '새우탕',
 '설렁탕',
 '순댓국',
 '어묵국',
 '오징어 국',
 '육개장',
 '장어탕',
 '재첩 국',
 '조갯국',
 '콩나물국',
 '토란국',
 '해장국',
 '갈치 찌개',
 '감자탕',
 '김치전골',
 '김치찌개',
 '꽃게탕',
 '낙지

In [45]:
food = food_lst + food_tmp

In [46]:
food_lst = list(set(food)) # 중복 제거
food_lst

['피자',
 '대구볼찜',
 '코다리찜',
 '앙금빵',
 '내장전골',
 '애호박전',
 '해신탕',
 '메밀소바',
 '푸딩',
 '고추장아찌',
 '옥수수구이',
 '곤약무침',
 '햄채소볶음',
 '파래 전',
 '낙지무침',
 '돌나물무침',
 '캘리포니아롤',
 '매실장아찌',
 '깐쇼새우',
 '우엉조림',
 '주꾸미삼겹살',
 '병어찜',
 '생선전',
 '무나물무침',
 '배추 겉절이',
 '불고기전골',
 '닭곰탕',
 '함박스테이크',
 '완자전',
 '멘보샤',
 '샐러드',
 '꼬치 구치',
 '양파링튀김',
 '오이무침',
 '연포탕',
 '방풍나물무침',
 '전어찜',
 '단호박찜',
 '해물콩나물찜',
 '송편',
 '뱅어포구이',
 '깨죽',
 '취나물무침',
 '찰밥',
 '경단',
 '미음',
 '돼지고기덮밥',
 '올갱이국',
 '돼지껍질 구이',
 '황태구이',
 '돼지등갈비찜',
 '마테차',
 '풋고추된장무침',
 '닭볶음탕',
 '동태조림',
 '우동볶음',
 '전복죽',
 '총각김치',
 '꽃게탕',
 '고사리나물무침',
 '갓김치',
 '갑오징어무침',
 '톳나물무침',
 '장어구이',
 '꽈리고추찜',
 '설렁탕',
 '순두부찌개',
 '선짓국',
 '재첩 국',
 '묵국',
 '미숫가루',
 '추로스',
 '증편',
 '현미죽',
 '샤부샤부',
 '볶음밥',
 '꼬막찜',
 '곰장어 구이',
 '주꾸미 삼겹살',
 '풋고추볶음',
 '라떼',
 '고구마 전',
 '우롱차',
 '덮밥',
 '노각무침',
 '새우튀김 롤',
 '꿀떡',
 '닭죽',
 '라조기',
 '김치볶음',
 '버섯구이표고버섯',
 '북어찜',
 '미소된장국',
 '알탕',
 '비빔막국수',
 '피조개 무침',
 '에이드',
 '가지볶음',
 '게국지',
 '기장밥',
 '참치 회 덮밥',
 '두릅무침',
 '캔디',
 '소라 산적',
 '오리고기로 스',
 '무말랭이무침',
 '북어 강정',
 '김구이',
 '김치전',
 '마늘종 볶음'

In [47]:
all_food = [word for word in food_lst if len(word) > 1]

- 빠진 음식이름 추가

In [48]:
food_add = ['펩시콜라','똠얌꿍','닭']

In [49]:
all_food.extend(food_add)

In [50]:
all_food.remove('파이')

In [51]:
if '닭' in all_food:
    print("True")
else:
    print("False")

True


In [93]:
train['food'] =train['사장댓글'].apply(lambda x: next((food for food in all_food if food in x), " "))

In [94]:
train.head()

,가게이름,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글,메뉴,store,branch,food
0,1988응답하라추억의옛날도시락-방이점,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,5,5.0,5.0,5.0,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,[직화]삼겹옛날도시락,,,
1,1988응답하라추억의옛날도시락-방이점,기름 있는 부분이 생각보다 많아요! 참고하세요,5,5.0,5.0,5.0,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,"[직화]삼겹옛날도시락,볶음김치",,,
2,1988응답하라추억의옛날도시락-방이점,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,5,5.0,5.0,5.0,늘 최고의 메뉴를 제공해드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,"[직화]삼겹옛날도시락,우삼겹청국장찌개＋대접밥",,,
3,1988응답하라추억의옛날도시락-방이점,".,.",5,5.0,5.0,5.0,저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,삼겹정찬도시락,,,
4,1988응답하라추억의옛날도시락-방이점,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,5,5.0,4.0,5.0,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,"갈비옛날도시락＋미니냉면＋（서비스）콜라,스팸옛날도시락＋김찌 정식＋（서비스）콜라,부드러...",,,


##6_긍/부정 라벨 컬럼화

In [98]:
import pandas as pd
from transformers import pipeline

def analyze_reviews(reviews):
    # 모델 불러오기
    model_name = "MLOpsEngineer/review-rate-prediction"
    classifier = pipeline("sentiment-analysis", model=model_name)

    results = []
    for review_text in tqdm(reviews):
        # 모델 추론
        result = classifier(review_text)[0]

        # 결과 해석
        label = result['label']
        score = result['score']

        if label in ['LABEL_0', 'LABEL_1']:
            sentiment = 0 # '부정'
        else:
            sentiment = 1 # '긍정'

        results.append((review_text, sentiment, score))

    # 데이터프레임 생성
    tmp = pd.DataFrame(results, columns=['review', 'sentiment', 'score'])
    return tmp

In [ ]:
sentiment_train = analyze_reviews(train['고객리뷰'])
sentiment_train

  0%|          | 0/37140 [00:00<?, ?it/s]

- Sentiment(긍/부정 사전학습 결과) 비율

In [ ]:
for i in range(0, 2):
    num_rating = sentiment_train[sentiment_train['sentiment'] == i].shape[0]
    print(f"별점 {i}점 개수: {num_rating}, {num_rating / sentiment_train.shape[0] * 100}%")

In [ ]:
train = pd.concat([train, sentiment_train], axis=1)
train.head()

In [106]:
train = train[['고객리뷰','사장댓글','sentiment','store','branch','food']]
train.head()

,고객리뷰,사장댓글,sentiment,store,branch,food
0,맛있게 잘먹었습니다,감사합니다! 다음번에는 다른것도 드셔보세요! 저희 다음번에 또 뵈용,1,,,
1,배달도 빠르고 가족모두 너무 맛나게 잘먹었어요.,"안녕하세요 탕화쿵푸 가오점입니다, 맛있게 드시고 소중한 별점 가득 남겨주셔서 감합니...",1,탕화,,
2,오늘도 정말 맛있습니다!,안녕하세요 맛있게 드셨다니 정말 기쁩니다 멋진 칭찬 남겨주셔서 넘 감사드립니다 항상...,1,,,
3,잘먹었습니다~!,고객님 안녕하세요! 소중한 리뷰 너무 감사드려요 더욱 더 만족스러운 식사를 하실수 ...,1,대한냉면,,냉면
4,늘 맛있게 잘 먹고 있어요 페페로니 너무 맛있음,안녕하세요! 토핑을 아끼지 않는 미.친피자입니다! 미.친피자 주문 주시고 리뷰로 찾...,1,미.친피자,,피자


In [ ]:
train.to_csv(f"{DATA_PATH}train+sentiment_0913.csv",index=False) # 저장

- train_data의 sentiment 기준 1:1 비율

In [107]:
shuffled_train = train.sample(frac=1, random_state=SEED) # 데이터 랜덤 섞기
ps_df = shuffled_train.loc[shuffled_train.sentiment == 1][:2041]
ng_df = shuffled_train.loc[shuffled_train.sentiment == 0]
df_train = pd.concat([ps_df,ng_df]).sample(frac=1,random_state=SEED).reset_index(drop=True)
df_train

,고객리뷰,사장댓글,sentiment,store,branch,food
0,양 맛 둘다 대 만족이에요 족발이 보쌈 대비 차가워요,주문해주셔서감사합니다 항상 신선한재료 차별화된 서비스로 노력 하겠습니다 담에 또 찾...,1,,,
1,족발 딱 제 스타일이예요. 서비스로 주신 치즈불닭이 너무 맛있네요.번창하세요,"고객님은 홍스스타일! 그리고 치즈불닭이아니고,치즈불족이라고 불러주세요!요!요! 어...",1,,,치즈
2,양도 맛도 모두 좋았습니다!,고객님 안녕하세요! 소중한 리뷰 너무 감사드려요 더욱 더 만족스러운 식사를 하실수 ...,1,대한냉면,,냉면
3,대통령 각하 만세.,주문 감사합니다~추운 날씨 감기조심하세요~,0,,,
4,"덜 끓여졌어요. 매번 마라탕은 여기서만 시키는데 배추, 푸주 익은 상태보니. 덜끓인...",믿고 주문해 주셨을텐데 아쉬움 드려 속상한 마음입니다. 재주문시에 남겨주신 피드백 ...,0,,,
...,...,...,...,...,...,...
4077,맛있게 잘먹었어요 ~!,감사합니다 고객님!,1,,,
4078,잘 먹었습니다.,고객님 주문해 주셔서 감사합니다 프루타의 신선달달한 과일과 디저트로 행복한 시간 되...,1,프루타,,과일
4079,맛있어요 처음시켜먹는데,맛있는 음식이 필요한 순간에 저희 떡형 복대점 을 선택해주셔서 대단히 감사드립니다 ...,1,떡형,,
4080,일단 제일 좋아하는 된장이 너무 짜서 배달 음식 중에서 제일 별로인 것도 감점 요인...,고객님의 후기 및 사진을 바탕으로 저희가 더욱 좋은 요리를 만들어 보겠습니다.,0,,,


In [108]:
df_train['고객리뷰'] = df_train['고객리뷰'].astype(str)
df_train['사장댓글'] = df_train['사장댓글'].astype(str)

In [109]:
df_train.isnull().sum()

고객리뷰         0
사장댓글         0
sentiment    0
store        0
branch       0
food         0
dtype: int64

In [110]:
review, reply = df_train["고객리뷰"] , df_train["사장댓글"]
len(review) , len(reply)

(4082, 4082)

In [111]:
review[:5]

0                        양 맛 둘다 대 만족이에요 족발이 보쌈 대비 차가워요
1           족발 딱 제 스타일이예요. 서비스로 주신 치즈불닭이 너무 맛있네요.번창하세요
2                                      양도 맛도 모두 좋았습니다!
3                                           대통령 각하 만세.
4    덜 끓여졌어요. 매번 마라탕은 여기서만 시키는데 배추, 푸주 익은 상태보니. 덜끓인...
Name: 고객리뷰, dtype: object

# 단어 사전 만들기

- 문자 단위로 토큰화 후 문자 번호 부여.

In [112]:
PAD = 0
SOS = 1
EOS = 2

class WordVocab():
    def __init__(self):
        self.word2index = {'<PAD>': PAD,
                           '<SOS>': SOS,
                           '<EOS>': EOS,}
        self.word2count = {}
        self.index2word = {PAD : '<PAD>',
                           SOS : '<SOS>',
                           EOS : '<EOS>'}
        self.n_words = 3  # PAD, SOS, EOS 포함

    def add_sentence(self, sentence):
        for word in sentence.split(' '): # 공백 기준으로 분리
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [113]:
print(f'원문: {review[0]}')
lang = WordVocab()
lang.add_sentence(review[0])
print('단어사전: ', lang.word2index)

원문: 양 맛 둘다 대 만족이에요 족발이 보쌈 대비 차가워요
단어사전:  {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '양': 3, '맛': 4, '둘다': 5, '대': 6, '만족이에요': 7, '족발이': 8, '보쌈': 9, '대비': 10, '차가워요': 11}


# 데이터셋 만들기

In [21]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 25.3 MB/s eta 0:00:00


In [22]:
from konlpy.tag import Okt

In [23]:
class TextDataset(Dataset):
    def __init__(self, df, min_length=3, max_length=32):
        super().__init__()

        # TOKEN 정의
        self.PAD = 0
        self.SOS = 1
        self.EOS = 2

        self.tagger = Okt() # 트위터 기반 형태소 분석기
        self.max_length = max_length

        # 형태소 처리와 단어사전 정의가 된 토큰을 담은 리스트
        src_clean = []
        tgt_clean = []

        # 단어 사전 생성
        wordvocab = WordVocab()

        for _, row in df.iterrows():
            # df 에서 'src'와 'tgt' 정의
            src = row['고객리뷰']
            tgt = row['사장댓글']
            store_name = row['store']
            food_name = row['food']
            branch_name = row['branch']

            input_data = f"{store_name} {branch_name} {food_name} {src}"
            target_data = tgt

            # 형태소 처리
            input_data = self.clean_text(input_data)
            target_data = self.clean_text(target_data)

            if len(input_data.split()) > min_length and len(target_data.split()) > min_length:
                # 최소 길이를 넘어가는 문장의 단어만 추가
                wordvocab.add_sentence(input_data)
                wordvocab.add_sentence(target_data)
                src_clean.append(input_data)
                tgt_clean.append(target_data)

            # 형태소 처리
            # src = self.clean_text(src)
            # tgt = self.clean_text(tgt)

            # if len(src.split()) > min_length and len(tgt.split()) > min_length:
            #     # 최소 길이를 넘어가는 문장의 단어만 추가
            #     wordvocab.add_sentence(src)
            #     wordvocab.add_sentence(tgt)
            #     src_clean.append(src)
            #     tgt_clean.append(tgt)

        self.srcs = src_clean
        self.tgts = tgt_clean
        self.wordvocab = wordvocab

    def clean_text(self, sentence):
        # 형태소 처리
        sentence = self.tagger.morphs(sentence)
        sentence = ' '.join(sentence)
        sentence = sentence.lower()
        return sentence

    def texts_to_sequences(self, sentence):
        # 문장 -> 시퀀스로 변환
        return [self.wordvocab.word2index[w] for w in sentence.split()]

    def pad_sequence(self, sentence_tokens):
        # 문장의 맨 끝 토큰은 제거
        sentence_tokens = sentence_tokens[:(self.max_length-1)]
        token_length = len(sentence_tokens)

        # 문장의 맨 끝부분에 <EOS> 토큰 추가
        sentence_tokens.append(self.EOS)

        for i in range(token_length, (self.max_length-1)):
            # 나머지 빈 곳은 <PAD> 토큰으로 채우기
            sentence_tokens.append(self.PAD)
        return sentence_tokens

    def __len__(self):
        return len(self.srcs)

    def __getitem__(self, idx):
        inputs = self.srcs[idx]
        inputs_sequences = self.texts_to_sequences(inputs) # 시퀀스로 바꾸고
        inputs_padded = self.pad_sequence(inputs_sequences) # pad 토큰 채우기

        outputs = self.tgts[idx]
        outputs_sequences = self.texts_to_sequences(outputs)
        outputs_padded = self.pad_sequence(outputs_sequences)

        return torch.tensor(inputs_padded), torch.tensor(outputs_padded)

In [24]:
MAX_LENGTH = 32
dataset = TextDataset(df_train, min_length=3, max_length=MAX_LENGTH)

In [ ]:
# MAX_LENGTH = 32
# train_dataset = TextDataset(df_train, min_length=3, max_length=MAX_LENGTH)
# test_dataset = TextDataset(df_test, min_length=3, max_length=MAX_LENGTH)

In [25]:
x, y = dataset[0]

In [26]:
print(f'x shape: {x.shape}')
print(x)

x shape: torch.Size([32])
tensor([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  2,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])


In [27]:
print(f'y shape: {y.shape}')
print(y)

y shape: torch.Size([32])
tensor([14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,  2,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])


In [28]:
from torch.utils.data import SubsetRandomSampler, random_split

train_size = int(len(dataset) * 0.8) # 80%의 데이터를 train에 할당
test_size = len(dataset) - train_size # 나머지 20% 데이터를 test에 할당
print(train_size, test_size)
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

2958 740


In [29]:
len(train_dataset)

2958

In [30]:
len(test_dataset)

740

## 데이터로더

In [31]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=True)

In [32]:
batch = next(iter(train_loader))
batch

[tensor([[  643,    45,   109,  3069,  3070,   219,    47,   351,    68,   175,
            136,   194,     2,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0],
         [   17,  1608, 13524,  1182,    74, 13525,     2,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0]]),
 tensor([[ 175,  522,  232,   35,  205, 3065,  165, 1959,  175, 3066,  157, 3067,
            35,  922,  585,  643,  204,  191,   29, 3068,   35, 3069, 3070,  262,
            93,  145,   66,    2,    0,    0,    0,    0],
         [  18,   19,   24,  157,  388,   66,   32,  927,   10, 1661,  144,  292,
            77,  175, 2284,   25,  388, 1307,   47,   90,   91,   80, 1259,  370,
           114, 1127,  189,   74,  163,   47,   76,    2]])]

In [33]:
# 1개의 배치 데이터를 추출
x, y = next(iter(train_loader))

# shape: (batch_size, sequence_length)
x.shape, y.shape

(torch.Size([2, 32]), torch.Size([2, 32]))

# Encoder 만들기

In [76]:
class Encoder(nn.Module):
    def __init__(self, num_embeddings, hidden_size, embedding_dim, num_layers):
        super().__init__()
        self.num_embeddings = num_embeddings # vocab size

        # 임베딩 레이어 정의 (number of vocabs, embedding dimension)
        self.emb_layer = nn.Embedding(num_embeddings, embedding_dim)

        # GRU (embedding dimension)
        self.rnn_layer = nn.GRU(embedding_dim,
                          hidden_size,
                          num_layers=num_layers,
                          bidirectional=False)

    def forward(self, x):
        x = self.emb_layer(x).permute(1, 0, 2)
        outputs, hn = self.rnn_layer(x)
        return outputs, hn

# Decoder 만들기

In [77]:
class Decoder(nn.Module):
    def __init__(self, num_embeddings, hidden_size, embedding_dim, num_layers=1, dropout=0.2):
        super().__init__()
        self.num_embeddings = num_embeddings
        self.emb_layer = nn.Embedding(num_embeddings, embedding_dim)
        self.dropout = nn.Dropout(dropout)
        self.rnn_layer = nn.GRU(embedding_dim,
                          hidden_size,
                          num_layers=num_layers,
                          batch_first=False) # (1, batch_size) 로 변환을 쉽게 하기 위함

        # 최종 출력은 단어사전의 개수(num_embeddings)
        self.fc_layer = nn.Linear(hidden_size, num_embeddings)

    def forward(self, x, hidden_state):
        x = x.unsqueeze(0) # (1, batch_size) 로 변환
        x = F.relu(self.emb_layer(x))
        x = self.dropout(x)
        outputs, hn = self.rnn_layer(x, hidden_state)
        outputs = self.fc_layer(outputs.squeeze(0)) # (sequence_length, batch_size, hidden_size(32))
        return outputs, hn

# Seq2Seq 모델 만들기

In [82]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, inputs, outputs, teacher_forcing_ratio=0.5):
        # inputs : (batch_size, sequence_length)
        # outputs: (batch_size, sequence_length)

        batch_size, output_length = outputs.shape
        output_num_vocabs = self.decoder.num_embeddings

        # 리턴할 예측된 outputs를 저장할 임시 변수
        # (sequence_length, batch_size, num_vocabs)
        predicted_outputs = torch.zeros(output_length, batch_size, output_num_vocabs).to(self.device)

        # 인코더에 입력 데이터 주입, encoder_output은 버리고 hidden_state 만 살림(Attention 안하니까 필요없음)
        # 여기서 hidden_state가 디코더에 주입할 context vector 가 됨
        # (number of layers(1), batch_size, hidden_size)
        _, decoder_hn = self.encoder(inputs)

        # (batch_size) shape의 SOS TOKEN으로 채워진 디코더 입력 생성
        decoder_input = torch.full((batch_size,), SOS, device=self.device)

        # 반복문으로 출력 단어를 생성
        # 0번째는 SOS TOKEN이 위치하므로, 1번째 인덱스부터 순회
        for t in range(0, output_length):
            # decoder_input : 디코더 입력 (batch_size) 형태의 SOS TOKEN로 채워진 입력
            # decoder_output: (batch_size, num_vocabs)
            # decoder_hidden: (number of layers(1), batch_size, hidden_size), context vector와 동일 shape
            decoder_output, decoder_hn = self.decoder(decoder_input, decoder_hn)

            # t번째 단어에 디코더의 output 저장
            predicted_outputs[t] = decoder_output

            # teacher forcing 적용 여부 확률로 결정
            # 정답치를 다음 RNN Cell의 입력으로 넣어주는 경우. 수렴속도가 빠를 수 있으나, 불안정할 수 있음
            teacher_force = random.random() < teacher_forcing_ratio

            # top1 단어 토큰 예측
            top1 = decoder_output.argmax(1)

            # teacher forcing 인 경우 ground truth 값을, 그렇지 않은 경우, 예측 값을 다음 input으로 지정
            decoder_input = outputs[:, t] if teacher_force else top1

        return predicted_outputs.permute(1, 0, 2) # (batch_size, sequence_length, num_vocabs)로 변경

In [87]:
# Encoder 정의
encoder = Encoder(num_embeddings=dataset.wordvocab.n_words,
                       hidden_size=32,
                       embedding_dim=64,
                       num_layers=1)
# Decoder 정의
decoder = Decoder(num_embeddings=dataset.wordvocab.n_words,
                       hidden_size=32,
                       embedding_dim=64,
                       num_layers=1)
# Seq2Seq 정의
seq2seq = Seq2Seq(encoder, decoder, device = "cpu")

In [84]:
x, y = next(iter(train_loader))
print(x.shape, y.shape)
# (batch_size, sequence_length), (batch_size, sequence_length)

torch.Size([2, 32]) torch.Size([2, 32])


In [88]:
output = seq2seq(x, y)
print(output.shape)
# (batch_size, sequence_length, num_vocabs)

torch.Size([2, 32, 15407])


## 모델 확인

In [89]:
NUM_VOCABS = dataset.wordvocab.n_words
HIDDEN_SIZE = 512
EMBEDDIMG_DIM = 256

print(f'num_vocabs: {NUM_VOCABS}\n======================')

# Encoder 정의
encoder = Encoder(num_embeddings=NUM_VOCABS,
                  hidden_size=HIDDEN_SIZE,
                  embedding_dim=EMBEDDIMG_DIM,
                  num_layers=1)
# Decoder 정의
decoder = Decoder(num_embeddings=NUM_VOCABS,
                  hidden_size=HIDDEN_SIZE,
                  embedding_dim=EMBEDDIMG_DIM,
                  num_layers=1)

# Seq2Seq 생성
# encoder, decoder를 device 모두 지정
model = Seq2Seq(encoder, decoder, device)
print(model)

num_vocabs: 15407
Seq2Seq(
  (encoder): Encoder(
    (emb_layer): Embedding(15407, 256)
    (rnn_layer): GRU(256, 512)
  )
  (decoder): Decoder(
    (emb_layer): Embedding(15407, 256)
    (dropout): Dropout(p=0.2, inplace=False)
    (rnn_layer): GRU(256, 512)
    (fc_layer): Linear(in_features=512, out_features=15407, bias=True)
  )
)


# 하이퍼파라미터 정의

In [41]:
class EarlyStopping:
    def __init__(self, patience=3, delta=0.0, mode='min', verbose=True):
        """
        patience (int): loss or score가 개선된 후 기다리는 기간. default: 3
        delta  (float): 개선시 인정되는 최소 변화 수치. default: 0.0
        mode     (str): 개선시 최소/최대값 기준 선정('min' or 'max'). default: 'min'.
        verbose (bool): 메시지 출력. default: True
        """
        self.early_stop = False
        self.patience = patience
        self.verbose = verbose
        self.counter = 0

        self.best_score = np.Inf if mode == 'min' else 0
        self.mode = mode
        self.delta = delta


    def __call__(self, score):

        if self.best_score is None:
            self.best_score = score
            self.counter = 0
        elif self.mode == 'min':
            if score < (self.best_score - self.delta):
                self.counter = 0
                self.best_score = score
                if self.verbose:
                    print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
            else:
                self.counter += 1
                if self.verbose:
                    print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                          f'Best: {self.best_score:.5f}' \
                          f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')

        elif self.mode == 'max':
            if score > (self.best_score + self.delta):
                self.counter = 0
                self.best_score = score
                if self.verbose:
                    print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
            else:
                self.counter += 1
                if self.verbose:
                    print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                          f'Best: {self.best_score:.5f}' \
                          f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')


        if self.counter >= self.patience:
            if self.verbose:
                print(f'[EarlyStop Triggered] Best Score: {self.best_score:.5f}')
            # Early Stop
            self.early_stop = True
        else:
            # Continue
            self.early_stop = False

In [90]:
LR = 1e-3
optimizer = optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss()
es = EarlyStopping(patience=5, delta=0.001, mode='min', verbose=True)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

# 학습 loop 만들기

In [93]:
def train(model, data_loader, optimizer, loss_fn, device):
    model.train()
    running_loss = 0

    for x, y in data_loader:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        # output: (batch_size, sequence_length, num_vocabs)
        model.to(device)
        output = model(x, y)
        output_dim = output.size(2)

        # 1번 index 부터 슬라이싱한 이유는 0번 index가 SOS TOKEN 이기 때문
        # (batch_size*sequence_length, num_vocabs) 로 변경
        output = output.reshape(-1, output_dim)

        # (batch_size*sequence_length) 로 변경
        y = y.view(-1)

        # Loss 계산
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x.size(0)

    return running_loss / len(data_loader)

In [95]:
def evaluate(model, data_loader, loss_fn, device):
    model.eval()

    eval_loss = 0

    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)
            output = model(x, y)
            output_dim = output.size(2)
            output = output.reshape(-1, output_dim)
            y = y.view(-1)

            # Loss 계산
            loss = loss_fn(output, y)

            eval_loss += loss.item() * x.size(0)

    return eval_loss / len(data_loader)

# 추론 함수(index2word)

In [96]:
def sequence_to_sentence(sequences, index2word):
    outputs = []
    for p in sequences:

        word = index2word[p]
        if p not in [SOS, EOS, PAD]:
            outputs.append(word)
        if word == EOS:
            break
    return ' '.join(outputs)

# 학습하기

In [99]:
device

'cuda'

In [101]:
# yogiyo_paraphrase
NUM_EPOCHS = 20
STATEDICT_PATH = '/content/drive/MyDrive/Final_Project/data/models/yogiyo_GRU_0913.pt'

best_loss = np.inf

for epoch in tqdm(range(NUM_EPOCHS), total=NUM_EPOCHS):
    loss = train(model, train_loader, optimizer, loss_fn, device)

    val_loss = evaluate(model, test_loader, loss_fn, device)

    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), STATEDICT_PATH)

    if epoch % 5 == 0:
        print(f'epoch: {epoch+1}, loss: {loss:.4f}, val_loss: {val_loss:.4f}')

    # Early Stop
    es(loss)
    if es.early_stop:
        break

    # Scheduler
    scheduler.step(val_loss)

model.load_state_dict(torch.load(STATEDICT_PATH))
torch.save(model.state_dict(), f'/content/drive/MyDrive/Final_Project/data/models/yogiyo_GRU_0913{best_loss:.4f}.pt')

  0%|          | 0/20 [00:00<?, ?it/s]

epoch: 1, loss: 8.3930, val_loss: 9.0760
[EarlyStopping] (Update) Best Score: 8.39303
[EarlyStopping] (Update) Best Score: 7.53949
[EarlyStopping] (Update) Best Score: 6.71851
[EarlyStopping] (Update) Best Score: 5.91606
Epoch 00004: reducing learning rate of group 0 to 5.0000e-04.
[EarlyStopping] (Update) Best Score: 4.73479
epoch: 6, loss: 4.1367, val_loss: 9.6048
[EarlyStopping] (Update) Best Score: 4.13669
[EarlyStopping] (Update) Best Score: 3.67795
Epoch 00007: reducing learning rate of group 0 to 2.5000e-04.
[EarlyStopping] (Update) Best Score: 3.13308
[EarlyStopping] (Update) Best Score: 2.84104
[EarlyStopping] (Update) Best Score: 2.59769
Epoch 00010: reducing learning rate of group 0 to 1.2500e-04.
epoch: 11, loss: 2.3249, val_loss: 10.1202
[EarlyStopping] (Update) Best Score: 2.32495
[EarlyStopping] (Update) Best Score: 2.16606
[EarlyStopping] (Update) Best Score: 2.01726
Epoch 00013: reducing learning rate of group 0 to 6.2500e-05.
[EarlyStopping] (Update) Best Score: 1.884

# 테스트 함수 만들기

- BLEU 점수

In [102]:
from nltk.translate.bleu_score import sentence_bleu

def calculate_bleu(text, generated_text):
    y = [text]  # 실제 데이터 y
    X = generated_text  # 생성된 텍스트 X

    # BLEU 점수 계산
    bleu_score = sentence_bleu(y, X)
    return bleu_score

- Paraphrase

In [103]:
from transformers import pipeline
pipe = pipeline("text2text-generation", model="lcw99/t5-base-korean-paraphrase")

def paraphrase_text(text):
    generated_text = pipe(text, max_length=128, early_stopping=True,
                          num_beams=4,
                          num_return_sequences=1,
                          no_repeat_ngram_size=4,
                          top_k=5,
                          top_p=0.95)
    generated_text = generated_text[0]['generated_text']
    return generated_text

In [104]:
# 10개만 보기
from torch.utils.data import SubsetRandomSampler

def random_evaluation_10(model, dataset, index2word, device, n=10):

    n_samples = len(dataset)
    indices = list(range(n_samples))
    np.random.shuffle(indices)      # Shuffle
    sampled_indices = indices[:n]   # Sampling N indices

    # 샘플링한 데이터를 기반으로 DataLoader 생성
    sampler = SubsetRandomSampler(sampled_indices)
    sampled_dataloader = DataLoader(dataset, batch_size=10, sampler=sampler)

    model.eval()
    with torch.no_grad():
        for x, y in sampled_dataloader:
            x, y = x.to(device), y.to(device)
            output = model(x, y, teacher_forcing_ratio=0)
            # output: (number of samples, sequence_length, num_vocabs)

            preds = output.detach().cpu().numpy()
            x = x.detach().cpu().numpy()
            y = y.detach().cpu().numpy()

            for i in range(n):
                result_review = sequence_to_sentence(x[i], index2word)
                result_reply = sequence_to_sentence(y[i], index2word)
                result_pred = sequence_to_sentence(preds[i].argmax(1), index2word)
                print(f'리뷰    : {result_review}')
                print(f'답변    : {result_reply}')
                print(f'예측답변: {paraphrase_text(result_pred)}') # paraphrased text 반환
                print(f'BLEU점수: {calculate_bleu(result_reply, result_pred)}')
                print('==='*10)

# 결과 확인하기

In [105]:
STATEDICT_PATH = '/content/drive/MyDrive/Final_Project/data/models/yogiyo_GRU_0913.pt'
model.load_state_dict(torch.load(STATEDICT_PATH))
random_evaluation_10(model, test_dataset, dataset.wordvocab.index2word, device)

리뷰    : 맛있게 잘 먹었어요 . 좋네요
답변    : 안녕하세요 도우 라 이크 입니다 . 잊지 않고 후기 감사합니다 항상 만족 하실 수 있도록 최선 을 다 하겠습니다 . 찜 꼬옥 눌러주시고 언제 든지 찾아주세요 항상 좋은 일 가득하시고 건강하세요
예측답변: 저희 순 살 만 공격 봉명 점을 이용해 주셔서 감사합니다 :) 즐겁고 기쁜 식사 되셨나요?
BLEU점수: 0.14573311256343435
리뷰    : 배달 이 다 된 후 에도 계속 문자 가 오는 통 에 계속 문 을 열 려고 했는데 못 본 사이 에 음식 이 상 할까봐 문 이 단단히 닫힌 거
답변    : 솔직한 후기 감사합니다 말씀 해주신 내용 참고 해서 더 좋은 요리 만들어 보겠습니다 감사합니다
예측답변: 고객 님 , 소중한 후기 잘 써 주셔서 감사합니다. 행복한 하루 되세요.
BLEU점수: 0.16460199336957007
리뷰    : 양도 푸 짐 하고 맛 도 있어 지난번 에 이어 또 주문 했는데 한 번 먹을 때 마다 머리카락 이 약간 나오는 건 위생 에 문제 가 있어요 .
답변    : 혹시 라도 이런 일이 반복 되면 바로 신고 해 주시 면 감사하겠습니다 다음 에 또 기회 가 되면 그땐 꼭 전화 드릴게요 다 들 감기 조심하세요
예측답변: 고객 만족도를 높이기 위해 최선을 다해 노력하겠습니다. 감사합니다.
BLEU점수: 0.08593272624414826
리뷰    : 몸살 감기 로 하루 종일 아무 것 도 잘 먹겠습니다
답변    : 수상한 삼겹살 을 찾아 주셔 감사 드립니다 ! 고객 님 들께서 만족스런 식사 를 하셨으면 하는 마음 으로 정성 을 다해 준비 하고 있습니다 ~ 소중한 후기 남겨 주셔서 감사 드리며
예측답변: 저희가 맛과 청결을 가장 중요하게 생각하며 항상 가족 모두가 먹는 갈비찜을 만들겠습니다.
BLEU점수: 0.26435626182232735
리뷰    : 와우 빨리 먹을라고 주문 했는데 끓여 먹어야 하는거라서 놀라고 어떻게 끓여서 먹는지 설명 서 가 없 음 에 두 번 놀라고